<a href="https://colab.research.google.com/github/ridersw/Complete-Machine-Learning/blob/main/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import pandas as pd

In [ ]:
#Dataset- IRIS Flower Dataset
#url- https://archive.ics.uci.edu/ml/datasets/iris

In [2]:
CSV_COLUMN_NAMES = ['SepalLength','SepalWidth','PetalLength','PetalWidth','Species']
SPECIES = ['Setosa','Versicolor','Virginica']

In [3]:
train_path = tf.keras.utils.get_file('iris_training.csv','https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv')
test_path = tf.keras.utils.get_file('iris_test.csv','https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv')

train = pd.read_csv(train_path, names = CSV_COLUMN_NAMES, header = 0)
test = pd.read_csv(test_path, names = CSV_COLUMN_NAMES, header = 0)


16384/573 [=========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


In [4]:
train.head()


,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [5]:
train_y = train.pop('Species')
test_y = test.pop('Species')
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [6]:
train.shape

(120, 4)

### Since we have to feed the data in the batches, we use the input function

In [7]:
def input_fn(features, labels, training=True, batch_size = 256):
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  if training:
    dataset = dataset.shuffle(1000).repeat()
  
  return dataset.batch(batch_size)

#### As all the values in CSV are already in numerical form, we will directly append them to feature columns array for processing

In [8]:
my_feature_columns = []

#train.keys will give us columns easily
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [9]:
#build model- Deep Neural Network with 2 hidden layers with 30 and 10 hidden nodes each
#n_classes -> 3 types of flowers we have

classifier = tf.estimator.DNNClassifier(feature_columns=my_feature_columns, hidden_units=[30,10],n_classes=3)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp61kv3p31', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [10]:
classifier.train(input_fn=lambda: input_fn(train, train_y, training = True), steps = 5000)
#steps is kinda similar to epoch- It will specify that we will gr through the dataset till out count of data saw totals to 5000

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp61kv3p31/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.506824, step = 0
INFO:tensorflow:global_step/sec: 368.65
INFO:tensorflow:loss = 0.9845485, step = 100 (0.278 sec)
INFO:tensorflow:global_step/sec: 460.583
INFO:tensorflow:loss = 0.8741665, step = 200 (0.216 sec)
INFO:tensorflow:global_step/sec: 

In [13]:
eval_result = classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training= False))
print('\nTest Set Accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-12-22T02:00:21
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp61kv3p31/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.62375s
INFO:tensorflow:Finished evaluation at 2021-12-22-02:00:22
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.93333334, average_loss = 0.46188086, global_step = 5000, loss = 0.46188086
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmp61kv3p31/model.ckpt-5000

Test Set Accuracy: 0.933



In [14]:
#check the prediction using user input parameters

def input_fn(features, batch_size = 256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth','PetalLength', 'PetalWidth']
predict = {}

print('Please prompt the numeric values as prompted')
for feature in features:
  valid = True
  while valid:
    val = input(feature + ": ")
    if not val.isdigit():
      valid = False

  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda:input_fn(predict))
for pred_dict in predictions:
  print(pred_dict)
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]

  print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id], 100* probability))

Please prompt the numeric values as prompted
SepalLength: 1.2
SepalWidth: 2.3
PetalLength: 5.1
PetalWidth: 0.5
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp61kv3p31/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([-1.1581241 , -0.40095142,  0.29643863], dtype=float32), 'probabilities': array([0.13486451, 0.28756335, 0.5775721 ], dtype=float32), 'class_ids': array([2]), 'classes': array([b'2'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction is "Virginica" (57.8%)
